In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
# import seaborn as sn
import os
from sklearn.preprocessing import OneHotEncoder
import nltk

# Extracting recipes from csv

In [2]:
raw_interactions = pd.read_csv('./Dataset/RAW_interactions.csv')
raw_recipes = pd.read_csv('./Dataset/RAW_recipes.csv')

In [3]:
print(raw_interactions.shape)
raw_interactions.head()

(1132367, 5)


,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."


In [4]:
print(raw_recipes.shape)
raw_recipes.head()

(231637, 12)


,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [5]:
# merge recipes and interactions tables

dfinal = raw_interactions.merge(raw_recipes, how='inner', left_on='recipe_id', right_on='id')
print(dfinal.shape)
dfinal.head()

(1132367, 17)


,user_id,recipe_id,date,rating,review,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...,white bean green chile pepper soup,40893,495,1533,2002-09-21,"['weeknight', 'time-to-make', 'course', 'main-...","[204.8, 5.0, 9.0, 26.0, 24.0, 2.0, 10.0]",4,"['combine beans , onion , chilies , 1 / 2 teas...",easy soup for the crockpot.,"['great northern beans', 'yellow onion', 'dice...",9
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall...",white bean green chile pepper soup,40893,495,1533,2002-09-21,"['weeknight', 'time-to-make', 'course', 'main-...","[204.8, 5.0, 9.0, 26.0, 24.0, 2.0, 10.0]",4,"['combine beans , onion , chilies , 1 / 2 teas...",easy soup for the crockpot.,"['great northern beans', 'yellow onion', 'dice...",9
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...,devilicious cookie cake delights,44394,20,56824,2002-10-27,"['30-minutes-or-less', 'time-to-make', 'course...","[132.3, 11.0, 39.0, 5.0, 4.0, 11.0, 5.0]",5,"['blend together cake mix , oil and eggs', 'ad...",NaN,"[""devil's food cake mix"", 'vegetable oil', 'eg...",4
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...,baked potato toppings,85009,10,64342,2004-02-25,"['15-minutes-or-less', 'time-to-make', 'course...","[2786.2, 342.0, 134.0, 290.0, 161.0, 301.0, 42.0]",3,['pick whichever topping you want to use and c...,these toppings sure makes a nice change from p...,"['mayonnaise', 'salsa', 'cheddar cheese', 'ref...",13
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin...",baked potato toppings,85009,10,64342,2004-02-25,"['15-minutes-or-less', 'time-to-make', 'course...","[2786.2, 342.0, 134.0, 290.0, 161.0, 301.0, 42.0]",3,['pick whichever topping you want to use and c...,these toppings sure makes a nice change from p...,"['mayonnaise', 'salsa', 'cheddar cheese', 'ref...",13


# Change date format (from str to date format)

In [6]:
from datetime import datetime

dfinal['date'] = dfinal['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').date())

In [7]:
print(type(dfinal['date'].iloc[0]))

<class 'datetime.date'>


# Categorising of recipes
Using tags

In [8]:
def str_to_list(row):

    x = row[1:-1] # remove []
    x = x.split(',')
    x = [x.strip()[1:-1] for x in x]
        
    return x

In [9]:
df_tags = dfinal[['recipe_id','tags']]
df_tags.drop_duplicates(inplace=True)
df_tags.reset_index(drop=True,inplace=True) # reset index
df_tags.head()

C:\Users\Gi Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,recipe_id,tags
0,40893,"['weeknight', 'time-to-make', 'course', 'main-..."
1,44394,"['30-minutes-or-less', 'time-to-make', 'course..."
2,85009,"['15-minutes-or-less', 'time-to-make', 'course..."
3,120345,"['15-minutes-or-less', 'time-to-make', 'course..."
4,134728,"['60-minutes-or-less', 'time-to-make', 'main-i..."


In [10]:
df_tags['tag_list'] = df_tags['tags'].apply(lambda x: str_to_list(x))

C:\Users\Gi Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
df_tags.head()

,recipe_id,tags,tag_list
0,40893,"['weeknight', 'time-to-make', 'course', 'main-...","[weeknight, time-to-make, course, main-ingredi..."
1,44394,"['30-minutes-or-less', 'time-to-make', 'course...","[30-minutes-or-less, time-to-make, course, mai..."
2,85009,"['15-minutes-or-less', 'time-to-make', 'course...","[15-minutes-or-less, time-to-make, course, mai..."
3,120345,"['15-minutes-or-less', 'time-to-make', 'course...","[15-minutes-or-less, time-to-make, course, mai..."
4,134728,"['60-minutes-or-less', 'time-to-make', 'main-i...","[60-minutes-or-less, time-to-make, main-ingred..."


### Find unique tags

In [12]:
all_tags = list()
for tags in df_tags['tag_list']:
    for t in tags:
        if t not in all_tags:
            print(t)
            all_tags.append(t)

weeknight
time-to-make
course
main-ingredient
preparation
occasion
soups-stews
beans
vegetables
easy
crock-pot-slow-cooker
dietary
equipment
30-minutes-or-less
for-large-groups
5-ingredients-or-less
drop-cookies
desserts
lunch
oven
potluck
fall
finger-food
heirloom-historical
holiday-event
kid-friendly
picnic
spring
summer
winter
cakes
cookies-and-brownies
easter
chocolate
gifts
christmas
hanukkah
new-years
seasonal
comfort-food
independence-day
ramadan
rosh-hashana
valentines-day
taste-mood
to-go
number-of-servings
presentation
15-minutes-or-less
condiments-etc
inexpensive
3-steps-or-less
cuisine
north-american
low-protein
healthy
jams-and-preserves
fruit
american
low-fat
low-sodium
low-cholesterol
low-saturated-fat
healthy-2
northeastern-united-states
low-in-something
berries
raspberries
60-minutes-or-less
poultry
chicken
high-protein
high-in-something
meat
main-dish
beef
mexican
low-carb
lamb-sheep
african
moroccan
stove-top
spicy
4-hours-or-less
for-1-or-2
casseroles
southwestern-u

In [13]:
# Selected category
category = ['meat', 'seafood', 'salads', 'vegetables', 'soup', 'pasta', 'desserts', 'breads', 'burgers', 'beverages']

In [14]:
# Creating new column
df_tags['category'] = ""
df_tags

C:\Users\Gi Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,recipe_id,tags,tag_list,category
0,40893,"['weeknight', 'time-to-make', 'course', 'main-...","[weeknight, time-to-make, course, main-ingredi...",
1,44394,"['30-minutes-or-less', 'time-to-make', 'course...","[30-minutes-or-less, time-to-make, course, mai...",
2,85009,"['15-minutes-or-less', 'time-to-make', 'course...","[15-minutes-or-less, time-to-make, course, mai...",
3,120345,"['15-minutes-or-less', 'time-to-make', 'course...","[15-minutes-or-less, time-to-make, course, mai...",
4,134728,"['60-minutes-or-less', 'time-to-make', 'main-i...","[60-minutes-or-less, time-to-make, main-ingred...",
...,...,...,...,...
231632,470995,"['15-minutes-or-less', 'time-to-make', 'course...","[15-minutes-or-less, time-to-make, course, mai...",
231633,241491,"['course', 'main-ingredient', 'cuisine', 'prep...","[course, main-ingredient, cuisine, preparation...",
231634,257796,"['main-ingredient', 'preparation', 'occasion',...","[main-ingredient, preparation, occasion, poult...",
231635,72730,"['30-minutes-or-less', 'time-to-make', 'course...","[30-minutes-or-less, time-to-make, course, mai...",


In [15]:
for index, row in df_tags.iterrows():
    
    # if the recipe falls under multiple categories, use the first category found in the tag
    if df_tags.at[index, 'category'] == "": 
        
        for cat in category: 
            if cat in row['tags']:
                df_tags.at[index, 'category'] = cat    

In [16]:
df_tags

,recipe_id,tags,tag_list,category
0,40893,"['weeknight', 'time-to-make', 'course', 'main-...","[weeknight, time-to-make, course, main-ingredi...",soup
1,44394,"['30-minutes-or-less', 'time-to-make', 'course...","[30-minutes-or-less, time-to-make, course, mai...",desserts
2,85009,"['15-minutes-or-less', 'time-to-make', 'course...","[15-minutes-or-less, time-to-make, course, mai...",vegetables
3,120345,"['15-minutes-or-less', 'time-to-make', 'course...","[15-minutes-or-less, time-to-make, course, mai...",desserts
4,134728,"['60-minutes-or-less', 'time-to-make', 'main-i...","[60-minutes-or-less, time-to-make, main-ingred...",meat
...,...,...,...,...
231632,470995,"['15-minutes-or-less', 'time-to-make', 'course...","[15-minutes-or-less, time-to-make, course, mai...",seafood
231633,241491,"['course', 'main-ingredient', 'cuisine', 'prep...","[course, main-ingredient, cuisine, preparation...",
231634,257796,"['main-ingredient', 'preparation', 'occasion',...","[main-ingredient, preparation, occasion, poult...",meat
231635,72730,"['30-minutes-or-less', 'time-to-make', 'course...","[30-minutes-or-less, time-to-make, course, mai...",


In [17]:
df_tags['category'].value_counts()

desserts      42204
vegetables    40108
              37321
meat          36910
pasta         23523
breads        14536
beverages     11124
soup          10081
seafood        9679
salads         5872
burgers         279
Name: category, dtype: int64

### Drop recipes that do not fall in any category

In [18]:
df_tags = df_tags[df_tags['category'] != ""]
df_tags

,recipe_id,tags,tag_list,category
0,40893,"['weeknight', 'time-to-make', 'course', 'main-...","[weeknight, time-to-make, course, main-ingredi...",soup
1,44394,"['30-minutes-or-less', 'time-to-make', 'course...","[30-minutes-or-less, time-to-make, course, mai...",desserts
2,85009,"['15-minutes-or-less', 'time-to-make', 'course...","[15-minutes-or-less, time-to-make, course, mai...",vegetables
3,120345,"['15-minutes-or-less', 'time-to-make', 'course...","[15-minutes-or-less, time-to-make, course, mai...",desserts
4,134728,"['60-minutes-or-less', 'time-to-make', 'main-i...","[60-minutes-or-less, time-to-make, main-ingred...",meat
...,...,...,...,...
231629,492861,"['15-minutes-or-less', 'time-to-make', 'course...","[15-minutes-or-less, time-to-make, course, cui...",beverages
231630,273409,"['60-minutes-or-less', 'time-to-make', 'course...","[60-minutes-or-less, time-to-make, course, mai...",vegetables
231631,249924,"['time-to-make', 'course', 'main-ingredient', ...","[time-to-make, course, main-ingredient, cuisin...",meat
231632,470995,"['15-minutes-or-less', 'time-to-make', 'course...","[15-minutes-or-less, time-to-make, course, mai...",seafood


In [19]:
df_tags['category'].value_counts()

desserts      42204
vegetables    40108
meat          36910
pasta         23523
breads        14536
beverages     11124
soup          10081
seafood        9679
salads         5872
burgers         279
Name: category, dtype: int64

In [20]:
df_tags.drop(columns = ['tags', 'tag_list'], inplace=True)

C:\Users\Gi Han\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [21]:
df_tags

,recipe_id,category
0,40893,soup
1,44394,desserts
2,85009,vegetables
3,120345,desserts
4,134728,meat
...,...,...
231629,492861,beverages
231630,273409,vegetables
231631,249924,meat
231632,470995,seafood


# Extract nutritional Components

- from string to respective categories
- convert them to float type

In [22]:
df_nutri = dfinal[['recipe_id', 'name','n_ingredients','minutes','nutrition']]
df_nutri.drop_duplicates(inplace=True)
df_nutri.reset_index(drop=True,inplace=True)
df_nutri

C:\Users\Gi Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,recipe_id,name,n_ingredients,minutes,nutrition
0,40893,white bean green chile pepper soup,9,495,"[204.8, 5.0, 9.0, 26.0, 24.0, 2.0, 10.0]"
1,44394,devilicious cookie cake delights,4,20,"[132.3, 11.0, 39.0, 5.0, 4.0, 11.0, 5.0]"
2,85009,baked potato toppings,13,10,"[2786.2, 342.0, 134.0, 290.0, 161.0, 301.0, 42.0]"
3,120345,sugared raspberries,2,10,"[838.0, 1.0, 820.0, 0.0, 2.0, 0.0, 71.0]"
4,134728,kfc honey bbq strips,12,40,"[316.0, 4.0, 40.0, 37.0, 78.0, 4.0, 10.0]"
...,...,...,...,...,...
231632,470995,perfectly fried shrimps with garlic,3,8,"[397.0, 62.0, 0.0, 8.0, 10.0, 28.0, 1.0]"
231633,241491,maple apple baked beans,10,375,"[518.1, 29.0, 138.0, 26.0, 34.0, 35.0, 24.0]"
231634,257796,slow cooker garlic chicken with rosemary,9,440,"[566.2, 59.0, 12.0, 15.0, 86.0, 55.0, 3.0]"
231635,72730,cranberry peach maple relish,6,22,"[1941.7, 3.0, 1569.0, 3.0, 10.0, 1.0, 154.0]"


In [23]:
df_nutri[['calories','total fat (PDV)','sugar (PDV)','sodium (PDV)','protein (PDV)','saturated fat (PDV)','carbohydrates (PDV)']] = df_nutri.nutrition.str.split(",",expand=True)

C:\Users\Gi Han\Anaconda3\lib\site-packages\pandas\core\frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [24]:
df_nutri['calories'] =  df_nutri['calories'].apply(lambda x: x.replace('[','')) 
df_nutri['carbohydrates (PDV)'] =  df_nutri['carbohydrates (PDV)'].apply(lambda x: x.replace(']',''))

C:\Users\Gi Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Gi Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [25]:
df_nutri[['calories','total fat (PDV)','sugar (PDV)','sodium (PDV)','protein (PDV)','saturated fat (PDV)','carbohydrates (PDV)']] = df_nutri[['calories','total fat (PDV)','sugar (PDV)','sodium (PDV)','protein (PDV)','saturated fat (PDV)','carbohydrates (PDV)']].astype('float')

In [26]:
print(df_nutri.shape)
df_nutri.drop(['nutrition'], axis=1,inplace=True)
df_nutri.head()

(231637, 12)


,recipe_id,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV)
0,40893,white bean green chile pepper soup,9,495,204.8,5.0,9.0,26.0,24.0,2.0,10.0
1,44394,devilicious cookie cake delights,4,20,132.3,11.0,39.0,5.0,4.0,11.0,5.0
2,85009,baked potato toppings,13,10,2786.2,342.0,134.0,290.0,161.0,301.0,42.0
3,120345,sugared raspberries,2,10,838.0,1.0,820.0,0.0,2.0,0.0,71.0
4,134728,kfc honey bbq strips,12,40,316.0,4.0,40.0,37.0,78.0,4.0,10.0


In [27]:
df_nutri.columns

Index(['recipe_id', 'name', 'n_ingredients', 'minutes', 'calories',
       'total fat (PDV)', 'sugar (PDV)', 'sodium (PDV)', 'protein (PDV)',
       'saturated fat (PDV)', 'carbohydrates (PDV)'],
      dtype='object')

In [28]:
lg_nutri = df_nutri[['recipe_id', 'calories', 'total fat (PDV)', 'sugar (PDV)', 'sodium (PDV)', 'protein (PDV)',
                        'saturated fat (PDV)', 'carbohydrates (PDV)']]

nutri = ['calories', 'total fat (PDV)', 'sugar (PDV)', 'sodium (PDV)', 'protein (PDV)', 
         'saturated fat (PDV)', 'carbohydrates (PDV)']

for n in nutri:
    lg_nutri[n] = lg_nutri[n].apply(lambda x: np.log(x))
    
lg_nutri

C:\Users\Gi Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,recipe_id,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV)
0,40893,5.322034,1.609438,2.197225,3.258097,3.178054,0.693147,2.302585
1,44394,4.885072,2.397895,3.663562,1.609438,1.386294,2.397895,1.609438
2,85009,7.932434,5.834811,4.897840,5.669881,5.081404,5.707110,3.737670
3,120345,6.731018,0.000000,6.709304,-inf,0.693147,-inf,4.262680
4,134728,5.755742,1.386294,3.688879,3.610918,4.356709,1.386294,2.302585
...,...,...,...,...,...,...,...,...
231632,470995,5.983936,4.127134,-inf,2.079442,2.302585,3.332205,0.000000
231633,241491,6.250168,3.367296,4.927254,3.258097,3.526361,3.555348,3.178054
231634,257796,6.338947,4.077537,2.484907,2.708050,4.454347,4.007333,1.098612
231635,72730,7.571319,1.098612,7.358194,1.098612,2.302585,0.000000,5.036953


In [29]:
# replace -inf with 0 
lg_nutri.replace(-np.inf, float(0), inplace=True)
lg_nutri

C:\Users\Gi Han\Anaconda3\lib\site-packages\pandas\core\frame.py:4263: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


,recipe_id,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV)
0,40893,5.322034,1.609438,2.197225,3.258097,3.178054,0.693147,2.302585
1,44394,4.885072,2.397895,3.663562,1.609438,1.386294,2.397895,1.609438
2,85009,7.932434,5.834811,4.897840,5.669881,5.081404,5.707110,3.737670
3,120345,6.731018,0.000000,6.709304,0.000000,0.693147,0.000000,4.262680
4,134728,5.755742,1.386294,3.688879,3.610918,4.356709,1.386294,2.302585
...,...,...,...,...,...,...,...,...
231632,470995,5.983936,4.127134,0.000000,2.079442,2.302585,3.332205,0.000000
231633,241491,6.250168,3.367296,4.927254,3.258097,3.526361,3.555348,3.178054
231634,257796,6.338947,4.077537,2.484907,2.708050,4.454347,4.007333,1.098612
231635,72730,7.571319,1.098612,7.358194,1.098612,2.302585,0.000000,5.036953


In [30]:
lg_nutri.columns

Index(['recipe_id', 'calories', 'total fat (PDV)', 'sugar (PDV)',
       'sodium (PDV)', 'protein (PDV)', 'saturated fat (PDV)',
       'carbohydrates (PDV)'],
      dtype='object')

In [31]:
# rename lg_nutri columns
lg_nutri.rename(columns ={'calories': 'log calories', 'total fat (PDV)': 'log fat', 'sugar (PDV)': 'log sugar', 
                 'sodium (PDV)': 'log sodium', 'protein (PDV)': 'log protein', 'saturated fat (PDV)': 'log sat fat',
                 'carbohydrates (PDV)': 'log carbs'}, inplace=True)

lg_nutri

C:\Users\Gi Han\Anaconda3\lib\site-packages\pandas\core\frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


,recipe_id,log calories,log fat,log sugar,log sodium,log protein,log sat fat,log carbs
0,40893,5.322034,1.609438,2.197225,3.258097,3.178054,0.693147,2.302585
1,44394,4.885072,2.397895,3.663562,1.609438,1.386294,2.397895,1.609438
2,85009,7.932434,5.834811,4.897840,5.669881,5.081404,5.707110,3.737670
3,120345,6.731018,0.000000,6.709304,0.000000,0.693147,0.000000,4.262680
4,134728,5.755742,1.386294,3.688879,3.610918,4.356709,1.386294,2.302585
...,...,...,...,...,...,...,...,...
231632,470995,5.983936,4.127134,0.000000,2.079442,2.302585,3.332205,0.000000
231633,241491,6.250168,3.367296,4.927254,3.258097,3.526361,3.555348,3.178054
231634,257796,6.338947,4.077537,2.484907,2.708050,4.454347,4.007333,1.098612
231635,72730,7.571319,1.098612,7.358194,1.098612,2.302585,0.000000,5.036953


In [32]:
# Combine with recipe data
df_nutri = df_nutri.merge(lg_nutri, on='recipe_id')
df_nutri

,recipe_id,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV),log calories,log fat,log sugar,log sodium,log protein,log sat fat,log carbs
0,40893,white bean green chile pepper soup,9,495,204.8,5.0,9.0,26.0,24.0,2.0,10.0,5.322034,1.609438,2.197225,3.258097,3.178054,0.693147,2.302585
1,44394,devilicious cookie cake delights,4,20,132.3,11.0,39.0,5.0,4.0,11.0,5.0,4.885072,2.397895,3.663562,1.609438,1.386294,2.397895,1.609438
2,85009,baked potato toppings,13,10,2786.2,342.0,134.0,290.0,161.0,301.0,42.0,7.932434,5.834811,4.897840,5.669881,5.081404,5.707110,3.737670
3,120345,sugared raspberries,2,10,838.0,1.0,820.0,0.0,2.0,0.0,71.0,6.731018,0.000000,6.709304,0.000000,0.693147,0.000000,4.262680
4,134728,kfc honey bbq strips,12,40,316.0,4.0,40.0,37.0,78.0,4.0,10.0,5.755742,1.386294,3.688879,3.610918,4.356709,1.386294,2.302585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231632,470995,perfectly fried shrimps with garlic,3,8,397.0,62.0,0.0,8.0,10.0,28.0,1.0,5.983936,4.127134,0.000000,2.079442,2.302585,3.332205,0.000000
231633,241491,maple apple baked beans,10,375,518.1,29.0,138.0,26.0,34.0,35.0,24.0,6.250168,3.367296,4.927254,3.258097,3.526361,3.555348,3.178054
231634,257796,slow cooker garlic chicken with rosemary,9,440,566.2,59.0,12.0,15.0,86.0,55.0,3.0,6.338947,4.077537,2.484907,2.708050,4.454347,4.007333,1.098612
231635,72730,cranberry peach maple relish,6,22,1941.7,3.0,1569.0,3.0,10.0,1.0,154.0,7.571319,1.098612,7.358194,1.098612,2.302585,0.000000,5.036953


# Combine dataset 
Merge with user information

In [33]:
dfuser = dfinal[['user_id', 'date', 'review','rating','recipe_id', 'description']]
dfuser.drop_duplicates(inplace=True)
dfuser.reset_index(drop=True,inplace=True)
dfuser

C:\Users\Gi Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,user_id,date,review,rating,recipe_id,description
0,38094,2003-02-17,Great with a salad. Cooked on top of stove for...,4,40893,easy soup for the crockpot.
1,1293707,2011-12-21,"So simple, so delicious! Great for chilly fall...",5,40893,easy soup for the crockpot.
2,8937,2002-12-01,This worked very well and is EASY. I used not...,4,44394,NaN
3,126440,2010-02-27,I made the Mexican topping and took it to bunk...,5,85009,these toppings sure makes a nice change from p...
4,57222,2011-10-01,"Made the cheddar bacon topping, adding a sprin...",5,85009,these toppings sure makes a nice change from p...
...,...,...,...,...,...,...
1132362,417131,2007-09-05,I made this over the weekend and changed it up...,4,241491,i've been looking for a perfect baked bean rec...
1132363,648540,2010-09-06,"Fabulous! We are vegetarian, so I used vegetar...",5,241491,i've been looking for a perfect baked bean rec...
1132364,1269180,2011-08-07,Very tender and juicy - had trouble getting t...,4,257796,delicious and easy!
1132365,116593,2003-12-09,Another approach is to start making sauce with...,0,72730,great simple alternative to regular cranberry ...


In [34]:
df_combine = df_nutri.merge(dfuser, how='inner', on='recipe_id')
df_combine

,recipe_id,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),...,log sugar,log sodium,log protein,log sat fat,log carbs,user_id,date,review,rating,description
0,40893,white bean green chile pepper soup,9,495,204.8,5.0,9.0,26.0,24.0,2.0,...,2.197225,3.258097,3.178054,0.693147,2.302585,38094,2003-02-17,Great with a salad. Cooked on top of stove for...,4,easy soup for the crockpot.
1,40893,white bean green chile pepper soup,9,495,204.8,5.0,9.0,26.0,24.0,2.0,...,2.197225,3.258097,3.178054,0.693147,2.302585,1293707,2011-12-21,"So simple, so delicious! Great for chilly fall...",5,easy soup for the crockpot.
2,44394,devilicious cookie cake delights,4,20,132.3,11.0,39.0,5.0,4.0,11.0,...,3.663562,1.609438,1.386294,2.397895,1.609438,8937,2002-12-01,This worked very well and is EASY. I used not...,4,NaN
3,85009,baked potato toppings,13,10,2786.2,342.0,134.0,290.0,161.0,301.0,...,4.897840,5.669881,5.081404,5.707110,3.737670,126440,2010-02-27,I made the Mexican topping and took it to bunk...,5,these toppings sure makes a nice change from p...
4,85009,baked potato toppings,13,10,2786.2,342.0,134.0,290.0,161.0,301.0,...,4.897840,5.669881,5.081404,5.707110,3.737670,57222,2011-10-01,"Made the cheddar bacon topping, adding a sprin...",5,these toppings sure makes a nice change from p...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1132362,241491,maple apple baked beans,10,375,518.1,29.0,138.0,26.0,34.0,35.0,...,4.927254,3.258097,3.526361,3.555348,3.178054,417131,2007-09-05,I made this over the weekend and changed it up...,4,i've been looking for a perfect baked bean rec...
1132363,241491,maple apple baked beans,10,375,518.1,29.0,138.0,26.0,34.0,35.0,...,4.927254,3.258097,3.526361,3.555348,3.178054,648540,2010-09-06,"Fabulous! We are vegetarian, so I used vegetar...",5,i've been looking for a perfect baked bean rec...
1132364,257796,slow cooker garlic chicken with rosemary,9,440,566.2,59.0,12.0,15.0,86.0,55.0,...,2.484907,2.708050,4.454347,4.007333,1.098612,1269180,2011-08-07,Very tender and juicy - had trouble getting t...,4,delicious and easy!
1132365,72730,cranberry peach maple relish,6,22,1941.7,3.0,1569.0,3.0,10.0,1.0,...,7.358194,1.098612,2.302585,0.000000,5.036953,116593,2003-12-09,Another approach is to start making sauce with...,0,great simple alternative to regular cranberry ...


In [35]:
df_combine = df_tags.merge(df_combine, how='inner', on='recipe_id')
df_combine

,recipe_id,category,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),...,log sugar,log sodium,log protein,log sat fat,log carbs,user_id,date,review,rating,description
0,40893,soup,white bean green chile pepper soup,9,495,204.8,5.0,9.0,26.0,24.0,...,2.197225,3.258097,3.178054,0.693147,2.302585,38094,2003-02-17,Great with a salad. Cooked on top of stove for...,4,easy soup for the crockpot.
1,40893,soup,white bean green chile pepper soup,9,495,204.8,5.0,9.0,26.0,24.0,...,2.197225,3.258097,3.178054,0.693147,2.302585,1293707,2011-12-21,"So simple, so delicious! Great for chilly fall...",5,easy soup for the crockpot.
2,44394,desserts,devilicious cookie cake delights,4,20,132.3,11.0,39.0,5.0,4.0,...,3.663562,1.609438,1.386294,2.397895,1.609438,8937,2002-12-01,This worked very well and is EASY. I used not...,4,NaN
3,85009,vegetables,baked potato toppings,13,10,2786.2,342.0,134.0,290.0,161.0,...,4.897840,5.669881,5.081404,5.707110,3.737670,126440,2010-02-27,I made the Mexican topping and took it to bunk...,5,these toppings sure makes a nice change from p...
4,85009,vegetables,baked potato toppings,13,10,2786.2,342.0,134.0,290.0,161.0,...,4.897840,5.669881,5.081404,5.707110,3.737670,57222,2011-10-01,"Made the cheddar bacon topping, adding a sprin...",5,these toppings sure makes a nice change from p...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978894,249924,meat,rocky mount nc bbq chicken sandwiches,5,70,505.9,44.0,19.0,18.0,63.0,...,2.944439,2.890372,4.143135,3.663562,2.079442,226863,2009-06-19,"I made this without the bread, and added some ...",5,"sweet and sticky with a kick of heat, this eas..."
978895,249924,meat,rocky mount nc bbq chicken sandwiches,5,70,505.9,44.0,19.0,18.0,63.0,...,2.944439,2.890372,4.143135,3.663562,2.079442,39835,2010-05-15,This chicken was great (skipped the bread). My...,5,"sweet and sticky with a kick of heat, this eas..."
978896,249924,meat,rocky mount nc bbq chicken sandwiches,5,70,505.9,44.0,19.0,18.0,63.0,...,2.944439,2.890372,4.143135,3.663562,2.079442,496803,2011-08-21,I really loved this! I cut the recipe back to...,5,"sweet and sticky with a kick of heat, this eas..."
978897,470995,seafood,perfectly fried shrimps with garlic,3,8,397.0,62.0,0.0,8.0,10.0,...,0.000000,2.079442,2.302585,3.332205,0.000000,169430,2013-04-17,Just as the title promised this is a lovely sh...,5,succulent shrimps fried in olive oil with cara...


In [36]:
df_combine.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 978899 entries, 0 to 978898
Data columns (total 24 columns):
recipe_id              978899 non-null int64
category               978899 non-null object
name                   978898 non-null object
n_ingredients          978899 non-null int64
minutes                978899 non-null int64
calories               978899 non-null float64
total fat (PDV)        978899 non-null float64
sugar (PDV)            978899 non-null float64
sodium (PDV)           978899 non-null float64
protein (PDV)          978899 non-null float64
saturated fat (PDV)    978899 non-null float64
carbohydrates (PDV)    978899 non-null float64
log calories           978899 non-null float64
log fat                978899 non-null float64
log sugar              978899 non-null float64
log sodium             978899 non-null float64
log protein            978899 non-null float64
log sat fat            978899 non-null float64
log carbs              978899 non-null float64
user

# Drop rows:

* without review
* without description
* description length less than 3
* without name

In [37]:
# drop rows without review, description, name

df_combine = df_combine.dropna(subset=['review', 'description', 'name'])
df_combine.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 957630 entries, 0 to 978898
Data columns (total 24 columns):
recipe_id              957630 non-null int64
category               957630 non-null object
name                   957630 non-null object
n_ingredients          957630 non-null int64
minutes                957630 non-null int64
calories               957630 non-null float64
total fat (PDV)        957630 non-null float64
sugar (PDV)            957630 non-null float64
sodium (PDV)           957630 non-null float64
protein (PDV)          957630 non-null float64
saturated fat (PDV)    957630 non-null float64
carbohydrates (PDV)    957630 non-null float64
log calories           957630 non-null float64
log fat                957630 non-null float64
log sugar              957630 non-null float64
log sodium             957630 non-null float64
log protein            957630 non-null float64
log sat fat            957630 non-null float64
log carbs              957630 non-null float64
user

In [38]:
# drop rows with descripion length less than 3

df_combine['description_token'] = df_combine['description'].apply(lambda x: x.strip().split())
df_combine.head()

C:\Users\Gi Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,recipe_id,category,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),...,log sodium,log protein,log sat fat,log carbs,user_id,date,review,rating,description,description_token
0,40893,soup,white bean green chile pepper soup,9,495,204.8,5.0,9.0,26.0,24.0,...,3.258097,3.178054,0.693147,2.302585,38094,2003-02-17,Great with a salad. Cooked on top of stove for...,4,easy soup for the crockpot.,"[easy, soup, for, the, crockpot.]"
1,40893,soup,white bean green chile pepper soup,9,495,204.8,5.0,9.0,26.0,24.0,...,3.258097,3.178054,0.693147,2.302585,1293707,2011-12-21,"So simple, so delicious! Great for chilly fall...",5,easy soup for the crockpot.,"[easy, soup, for, the, crockpot.]"
3,85009,vegetables,baked potato toppings,13,10,2786.2,342.0,134.0,290.0,161.0,...,5.669881,5.081404,5.707110,3.737670,126440,2010-02-27,I made the Mexican topping and took it to bunk...,5,these toppings sure makes a nice change from p...,"[these, toppings, sure, makes, a, nice, change..."
4,85009,vegetables,baked potato toppings,13,10,2786.2,342.0,134.0,290.0,161.0,...,5.669881,5.081404,5.707110,3.737670,57222,2011-10-01,"Made the cheddar bacon topping, adding a sprin...",5,these toppings sure makes a nice change from p...,"[these, toppings, sure, makes, a, nice, change..."
5,120345,desserts,sugared raspberries,2,10,838.0,1.0,820.0,0.0,2.0,...,0.000000,0.693147,0.000000,4.262680,52282,2005-05-21,very very sweet. after i waited the 2 days i b...,4,here's an old method for preserving fruit with...,"[here's, an, old, method, for, preserving, fru..."


In [40]:
df_combine['description_len'] = df_combine['description_token'].str.len()
df_combine

C:\Users\Gi Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,recipe_id,category,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),...,log protein,log sat fat,log carbs,user_id,date,review,rating,description,description_token,description_len
0,40893,soup,white bean green chile pepper soup,9,495,204.8,5.0,9.0,26.0,24.0,...,3.178054,0.693147,2.302585,38094,2003-02-17,Great with a salad. Cooked on top of stove for...,4,easy soup for the crockpot.,"[easy, soup, for, the, crockpot.]",5
1,40893,soup,white bean green chile pepper soup,9,495,204.8,5.0,9.0,26.0,24.0,...,3.178054,0.693147,2.302585,1293707,2011-12-21,"So simple, so delicious! Great for chilly fall...",5,easy soup for the crockpot.,"[easy, soup, for, the, crockpot.]",5
3,85009,vegetables,baked potato toppings,13,10,2786.2,342.0,134.0,290.0,161.0,...,5.081404,5.707110,3.737670,126440,2010-02-27,I made the Mexican topping and took it to bunk...,5,these toppings sure makes a nice change from p...,"[these, toppings, sure, makes, a, nice, change...",25
4,85009,vegetables,baked potato toppings,13,10,2786.2,342.0,134.0,290.0,161.0,...,5.081404,5.707110,3.737670,57222,2011-10-01,"Made the cheddar bacon topping, adding a sprin...",5,these toppings sure makes a nice change from p...,"[these, toppings, sure, makes, a, nice, change...",25
5,120345,desserts,sugared raspberries,2,10,838.0,1.0,820.0,0.0,2.0,...,0.693147,0.000000,4.262680,52282,2005-05-21,very very sweet. after i waited the 2 days i b...,4,here's an old method for preserving fruit with...,"[here's, an, old, method, for, preserving, fru...",98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978894,249924,meat,rocky mount nc bbq chicken sandwiches,5,70,505.9,44.0,19.0,18.0,63.0,...,4.143135,3.663562,2.079442,226863,2009-06-19,"I made this without the bread, and added some ...",5,"sweet and sticky with a kick of heat, this eas...","[sweet, and, sticky, with, a, kick, of, heat,,...",63
978895,249924,meat,rocky mount nc bbq chicken sandwiches,5,70,505.9,44.0,19.0,18.0,63.0,...,4.143135,3.663562,2.079442,39835,2010-05-15,This chicken was great (skipped the bread). My...,5,"sweet and sticky with a kick of heat, this eas...","[sweet, and, sticky, with, a, kick, of, heat,,...",63
978896,249924,meat,rocky mount nc bbq chicken sandwiches,5,70,505.9,44.0,19.0,18.0,63.0,...,4.143135,3.663562,2.079442,496803,2011-08-21,I really loved this! I cut the recipe back to...,5,"sweet and sticky with a kick of heat, this eas...","[sweet, and, sticky, with, a, kick, of, heat,,...",63
978897,470995,seafood,perfectly fried shrimps with garlic,3,8,397.0,62.0,0.0,8.0,10.0,...,2.302585,3.332205,0.000000,169430,2013-04-17,Just as the title promised this is a lovely sh...,5,succulent shrimps fried in olive oil with cara...,"[succulent, shrimps, fried, in, olive, oil, wi...",41


In [45]:
df_combine = df_combine[df_combine['description_len'] >= 3]
df_combine

,recipe_id,category,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),...,log protein,log sat fat,log carbs,user_id,date,review,rating,description,description_token,description_len
0,40893,soup,white bean green chile pepper soup,9,495,204.8,5.0,9.0,26.0,24.0,...,3.178054,0.693147,2.302585,38094,2003-02-17,Great with a salad. Cooked on top of stove for...,4,easy soup for the crockpot.,"[easy, soup, for, the, crockpot.]",5
1,40893,soup,white bean green chile pepper soup,9,495,204.8,5.0,9.0,26.0,24.0,...,3.178054,0.693147,2.302585,1293707,2011-12-21,"So simple, so delicious! Great for chilly fall...",5,easy soup for the crockpot.,"[easy, soup, for, the, crockpot.]",5
3,85009,vegetables,baked potato toppings,13,10,2786.2,342.0,134.0,290.0,161.0,...,5.081404,5.707110,3.737670,126440,2010-02-27,I made the Mexican topping and took it to bunk...,5,these toppings sure makes a nice change from p...,"[these, toppings, sure, makes, a, nice, change...",25
4,85009,vegetables,baked potato toppings,13,10,2786.2,342.0,134.0,290.0,161.0,...,5.081404,5.707110,3.737670,57222,2011-10-01,"Made the cheddar bacon topping, adding a sprin...",5,these toppings sure makes a nice change from p...,"[these, toppings, sure, makes, a, nice, change...",25
5,120345,desserts,sugared raspberries,2,10,838.0,1.0,820.0,0.0,2.0,...,0.693147,0.000000,4.262680,52282,2005-05-21,very very sweet. after i waited the 2 days i b...,4,here's an old method for preserving fruit with...,"[here's, an, old, method, for, preserving, fru...",98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978894,249924,meat,rocky mount nc bbq chicken sandwiches,5,70,505.9,44.0,19.0,18.0,63.0,...,4.143135,3.663562,2.079442,226863,2009-06-19,"I made this without the bread, and added some ...",5,"sweet and sticky with a kick of heat, this eas...","[sweet, and, sticky, with, a, kick, of, heat,,...",63
978895,249924,meat,rocky mount nc bbq chicken sandwiches,5,70,505.9,44.0,19.0,18.0,63.0,...,4.143135,3.663562,2.079442,39835,2010-05-15,This chicken was great (skipped the bread). My...,5,"sweet and sticky with a kick of heat, this eas...","[sweet, and, sticky, with, a, kick, of, heat,,...",63
978896,249924,meat,rocky mount nc bbq chicken sandwiches,5,70,505.9,44.0,19.0,18.0,63.0,...,4.143135,3.663562,2.079442,496803,2011-08-21,I really loved this! I cut the recipe back to...,5,"sweet and sticky with a kick of heat, this eas...","[sweet, and, sticky, with, a, kick, of, heat,,...",63
978897,470995,seafood,perfectly fried shrimps with garlic,3,8,397.0,62.0,0.0,8.0,10.0,...,2.302585,3.332205,0.000000,169430,2013-04-17,Just as the title promised this is a lovely sh...,5,succulent shrimps fried in olive oil with cara...,"[succulent, shrimps, fried, in, olive, oil, wi...",41


In [49]:
df_combine = df_combine.drop(['description_token', 'description_len'], axis=1)
df_combine

,recipe_id,category,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),...,log sugar,log sodium,log protein,log sat fat,log carbs,user_id,date,review,rating,description
0,40893,soup,white bean green chile pepper soup,9,495,204.8,5.0,9.0,26.0,24.0,...,2.197225,3.258097,3.178054,0.693147,2.302585,38094,2003-02-17,Great with a salad. Cooked on top of stove for...,4,easy soup for the crockpot.
1,40893,soup,white bean green chile pepper soup,9,495,204.8,5.0,9.0,26.0,24.0,...,2.197225,3.258097,3.178054,0.693147,2.302585,1293707,2011-12-21,"So simple, so delicious! Great for chilly fall...",5,easy soup for the crockpot.
3,85009,vegetables,baked potato toppings,13,10,2786.2,342.0,134.0,290.0,161.0,...,4.897840,5.669881,5.081404,5.707110,3.737670,126440,2010-02-27,I made the Mexican topping and took it to bunk...,5,these toppings sure makes a nice change from p...
4,85009,vegetables,baked potato toppings,13,10,2786.2,342.0,134.0,290.0,161.0,...,4.897840,5.669881,5.081404,5.707110,3.737670,57222,2011-10-01,"Made the cheddar bacon topping, adding a sprin...",5,these toppings sure makes a nice change from p...
5,120345,desserts,sugared raspberries,2,10,838.0,1.0,820.0,0.0,2.0,...,6.709304,0.000000,0.693147,0.000000,4.262680,52282,2005-05-21,very very sweet. after i waited the 2 days i b...,4,here's an old method for preserving fruit with...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978894,249924,meat,rocky mount nc bbq chicken sandwiches,5,70,505.9,44.0,19.0,18.0,63.0,...,2.944439,2.890372,4.143135,3.663562,2.079442,226863,2009-06-19,"I made this without the bread, and added some ...",5,"sweet and sticky with a kick of heat, this eas..."
978895,249924,meat,rocky mount nc bbq chicken sandwiches,5,70,505.9,44.0,19.0,18.0,63.0,...,2.944439,2.890372,4.143135,3.663562,2.079442,39835,2010-05-15,This chicken was great (skipped the bread). My...,5,"sweet and sticky with a kick of heat, this eas..."
978896,249924,meat,rocky mount nc bbq chicken sandwiches,5,70,505.9,44.0,19.0,18.0,63.0,...,2.944439,2.890372,4.143135,3.663562,2.079442,496803,2011-08-21,I really loved this! I cut the recipe back to...,5,"sweet and sticky with a kick of heat, this eas..."
978897,470995,seafood,perfectly fried shrimps with garlic,3,8,397.0,62.0,0.0,8.0,10.0,...,0.000000,2.079442,2.302585,3.332205,0.000000,169430,2013-04-17,Just as the title promised this is a lovely sh...,5,succulent shrimps fried in olive oil with cara...


### Filtering data
- User that has reviewed 500 <= reviews <= 1000 reviews

In [50]:
user_filter = df_combine.groupby("user_id").filter(lambda x: 500 <= len(x)  <= 1000)

In [51]:
user_filter

,recipe_id,category,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),...,log sugar,log sodium,log protein,log sat fat,log carbs,user_id,date,review,rating,description
5,120345,desserts,sugared raspberries,2,10,838.0,1.0,820.0,0.0,2.0,...,6.709304,0.000000,0.693147,0.000000,4.262680,52282,2005-05-21,very very sweet. after i waited the 2 days i b...,4,here's an old method for preserving fruit with...
16,200236,meat,lamb stew with tomatoes chickpeas and spices,16,150,606.5,65.0,12.0,34.0,65.0,...,2.484907,3.526361,4.174387,4.418841,1.945910,68960,2006-12-19,We really enjoyed this and so did both my youn...,4,north african spices with a basic meat stew re...
21,225241,vegetables,chicken tamale pie for 2 ww core,16,45,248.0,4.0,28.0,21.0,44.0,...,3.332205,3.044522,3.784190,1.098612,2.397895,353579,2007-08-14,"Very good, made with ingredients I usually hav...",5,after wondering how recipe #224699 would work ...
35,79222,soup,potato crab chowder,13,45,274.2,12.0,15.0,18.0,32.0,...,2.708050,2.890372,3.465736,3.091042,2.484907,446143,2008-02-01,"Oh, how wonderful! I doubled the crab, and ad...",4,soup for the soul!
57,321038,desserts,hg s grab n go breakfast cookies weight wat...,14,22,182.4,2.0,50.0,7.0,11.0,...,3.912023,1.945910,2.397895,0.000000,2.564949,539686,2011-03-28,My very health conscious daughter and I triple...,5,i received a hungry girl email today and this ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978869,286499,meat,salsa grilled chicken,4,60,345.4,11.0,25.0,74.0,108.0,...,3.218876,4.304065,4.682131,1.945910,1.609438,844554,2012-06-06,Very good recipe! I did have to use salsa verd...,5,very tasty and calorie friendly! use your favo...
978871,40514,vegetables,sun dried tomato bruschetta with goat cheese,5,15,94.6,5.0,4.0,8.0,8.0,...,1.386294,2.079442,2.079442,2.302585,1.098612,469903,2008-06-24,Simple flavors of summertime! Instead of bread...,4,these are good. if you haven't tried goat chee...
978882,190698,salads,strawberry balsamic vinaigrette,4,5,102.1,15.0,8.0,0.0,0.0,...,2.079442,0.000000,0.000000,1.791759,0.000000,145352,2007-05-14,Oh I loved this! So quick and so tasty! I made...,5,saw this on rachel rey 30 min meals! she said ...
978886,190698,salads,strawberry balsamic vinaigrette,4,5,102.1,15.0,8.0,0.0,0.0,...,2.079442,0.000000,0.000000,1.791759,0.000000,333017,2009-06-24,"This was just OK. When I first tasted it, I fo...",3,saw this on rachel rey 30 min meals! she said ...


In [52]:
user_filter['user_id'].value_counts()

61660     982
56003     975
145352    968
537937    954
171790    952
         ... 
18391     511
37305     508
269480    507
228458    506
135566    501
Name: user_id, Length: 123, dtype: int64

### Sort dataframe by date

In [53]:
final = user_filter.sort_values(by="date")
final.drop_duplicates(inplace=True)
print(final.shape)
final.head()

(85237, 24)


,recipe_id,category,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),...,log sugar,log sodium,log protein,log sat fat,log carbs,user_id,date,review,rating,description
192089,8661,salads,oliveau salad dressing,8,20,168.7,28.0,3.0,3.0,0.0,...,1.098612,1.098612,0.000000,2.484907,0.000000,13483,2001-08-05,Good recipe and easy to make. I found it a li...,4,traditional french dressing -- this recipe was...
168111,10859,vegetables,broccoli cauliflower and rice casserole,5,85,228.8,24.0,12.0,46.0,18.0,...,2.484907,3.828641,2.890372,3.465736,1.386294,8629,2001-08-15,This is a nice casserole. It is very easy to ...,2,easy and quick to prepare. grandmother's recip...
449496,902,vegetables,microwaved garlic mushroom rolls,7,0,1188.6,88.0,53.0,69.0,75.0,...,3.970292,4.234107,4.317488,5.030438,3.828641,13483,2001-08-16,Very easy to make for a lunch. I served it co...,3,"i use brown flat mushrooms and slice them, i f..."
567340,9115,salads,mustard pepper dressing,8,5,381.5,48.0,69.0,9.0,10.0,...,4.234107,2.197225,2.302585,3.433987,1.791759,13483,2001-09-04,For my taste I added a little extra garlic to ...,5,yummy dressing with a green salad
831211,8808,pasta,bbq chicken with a zing,8,35,309.3,24.0,39.0,17.0,55.0,...,3.663562,2.833213,4.007333,2.397895,1.386294,13483,2001-09-04,"The ""secret"" to this recipe, I believe, is sav...",4,this is a tasty chicken dish that you can eith...


In [54]:
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85237 entries, 192089 to 246716
Data columns (total 24 columns):
recipe_id              85237 non-null int64
category               85237 non-null object
name                   85237 non-null object
n_ingredients          85237 non-null int64
minutes                85237 non-null int64
calories               85237 non-null float64
total fat (PDV)        85237 non-null float64
sugar (PDV)            85237 non-null float64
sodium (PDV)           85237 non-null float64
protein (PDV)          85237 non-null float64
saturated fat (PDV)    85237 non-null float64
carbohydrates (PDV)    85237 non-null float64
log calories           85237 non-null float64
log fat                85237 non-null float64
log sugar              85237 non-null float64
log sodium             85237 non-null float64
log protein            85237 non-null float64
log sat fat            85237 non-null float64
log carbs              85237 non-null float64
user_id            

In [55]:
# Convert to csv
final.to_csv('./Dataset/final_dataset.csv', index=False)

# Train & Test Dataset

In [56]:
all_dataset = pd.read_csv('./Dataset/final_dataset.csv')
all_dataset.head()

,recipe_id,category,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),...,log sugar,log sodium,log protein,log sat fat,log carbs,user_id,date,review,rating,description
0,8661,salads,oliveau salad dressing,8,20,168.7,28.0,3.0,3.0,0.0,...,1.098612,1.098612,0.000000,2.484907,0.000000,13483,2001-08-05,Good recipe and easy to make. I found it a li...,4,traditional french dressing -- this recipe was...
1,10859,vegetables,broccoli cauliflower and rice casserole,5,85,228.8,24.0,12.0,46.0,18.0,...,2.484907,3.828641,2.890372,3.465736,1.386294,8629,2001-08-15,This is a nice casserole. It is very easy to ...,2,easy and quick to prepare. grandmother's recip...
2,902,vegetables,microwaved garlic mushroom rolls,7,0,1188.6,88.0,53.0,69.0,75.0,...,3.970292,4.234107,4.317488,5.030438,3.828641,13483,2001-08-16,Very easy to make for a lunch. I served it co...,3,"i use brown flat mushrooms and slice them, i f..."
3,9115,salads,mustard pepper dressing,8,5,381.5,48.0,69.0,9.0,10.0,...,4.234107,2.197225,2.302585,3.433987,1.791759,13483,2001-09-04,For my taste I added a little extra garlic to ...,5,yummy dressing with a green salad
4,8808,pasta,bbq chicken with a zing,8,35,309.3,24.0,39.0,17.0,55.0,...,3.663562,2.833213,4.007333,2.397895,1.386294,13483,2001-09-04,"The ""secret"" to this recipe, I believe, is sav...",4,this is a tasty chicken dish that you can eith...


In [57]:
def train_test_split(df):

    import math

    date = df.iloc[math.floor(.75 * len(df))]['date']
    print('Date of the 75th-percentile review: ', date)  

    i_date = df.index[df['date'] == date].tolist()
    print(f'Index of the first review on {date}: ', i_date[0]) 

    train_set = df.iloc[:i_date[0]]
    print(train_set.shape)

    test_set = df.iloc[i_date[0]:]
    print(test_set.shape)

    return (train_set, test_set)


In [58]:
train_set, test_set = train_test_split(all_dataset)

Date of the 75th-percentile review:  2010-04-15
Index of the first review on 2010-04-15:  63914
(63914, 24)
(21323, 24)


In [59]:
train_set.tail()

,recipe_id,category,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),...,log sugar,log sodium,log protein,log sat fat,log carbs,user_id,date,review,rating,description
63909,231897,meat,delicious grilled chicken marinade,6,5,755.8,111.0,39.0,136.0,3.0,...,3.663562,4.912655,1.098612,3.850148,2.197225,844554,2010-04-14,This marinade made the chicken SO-O-O tender i...,5,this is the most delicious marinade for large ...
63910,342629,salads,chicken egg potato salad,10,150,212.0,19.0,7.0,7.0,26.0,...,1.945910,1.945910,3.258097,2.944439,1.098612,50969,2010-04-14,"I made this recipe on 4/13/14 for the "" Soup-S...",4,as simple as combining my three favorite mayo-...
63911,382231,vegetables,super easy cucumber salad,3,1460,90.2,8.0,21.0,8.0,2.0,...,3.044522,2.079442,0.693147,1.386294,1.098612,464080,2010-04-14,"Just like mom used to make. Mmmm, mmmmm, good...",5,i can't remember where or how i learned to mak...
63912,406610,desserts,indoor s mores,3,5,174.5,9.0,74.0,2.0,3.0,...,4.304065,0.693147,1.098612,3.258097,2.197225,179133,2010-04-14,"Oh my goodness, I wish I'd thought of this mys...",5,brilliantly made up and named by my hubby on o...
63913,372962,breads,cinnamon maple scones,10,35,349.2,20.0,51.0,22.0,13.0,...,3.931826,3.091042,2.564949,3.637586,2.772589,178427,2010-04-14,Very tasty indeed! I don't use soy milk so I s...,5,scones that will beat starbucks' scones any day.


In [60]:
test_set.head()

,recipe_id,category,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),...,log sugar,log sodium,log protein,log sat fat,log carbs,user_id,date,review,rating,description
63914,327626,soup,miracle black bean veggie soup,16,55,187.5,4.0,19.0,13.0,18.0,...,2.944439,2.564949,2.890372,0.693147,2.302585,469903,2010-04-15,"Nicely spiced black bean soup, it certainly wo...",5,"this comfort-filled, nourishing soup is my mod..."
63915,89503,meat,crispy baked chicken strips,7,30,617.5,11.0,2.0,20.0,78.0,...,0.693147,2.995732,4.356709,2.079442,3.433987,125388,2010-04-15,"These were really good, and so easy to mix up ...",5,these are delicious! and much more cost-effect...
63916,418966,vegetables,tossed caprese salad,9,20,243.9,30.0,12.0,16.0,26.0,...,2.484907,2.772589,3.258097,3.737670,0.000000,464080,2010-04-15,Very good indeed. Great use for the Trader Jo...,5,a tossed salad version of a classic italian ap...
63917,414998,beverages,single serve hot chocolate fast,4,3,187.3,5.0,103.0,10.0,23.0,...,4.634729,2.302585,3.135494,1.386294,2.197225,201581,2010-04-15,What a great treat on a chilly morning. Easy t...,5,you're out of all the usual suspects and someb...
63918,161941,meat,smoked pork shoulder smoker or pit,15,90,735.0,7.0,564.0,738.0,18.0,...,6.335054,6.603944,2.890372,1.386294,4.077537,804550,2010-04-15,Delicious smokehouse flavor. When my DD and I ...,5,this recipe was given to me by a friend of my ...


## Check all users in test set is available in train set
Remove new user in test set

In [61]:
userid_train = train_set['user_id'].unique()
userid_train[:10]

array([13483,  8629, 18391, 21752, 22015, 20754, 27783,  6258, 28177,
       37305], dtype=int64)

In [62]:
userid_test = test_set['user_id'].unique()
userid_test[:10]

array([469903, 125388, 464080, 201581, 804550,  56003,   8629,  37779,
       211184, 171790], dtype=int64)

In [63]:
new_user = []
for i in userid_test:
    if i not in userid_train:
        new_user.append(i)
        
new_user

[1680722]

In [64]:
# Drop new user in test dataset
test_set = test_set[test_set['user_id'] != 1680722]
test_set

,recipe_id,category,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),...,log sugar,log sodium,log protein,log sat fat,log carbs,user_id,date,review,rating,description
63914,327626,soup,miracle black bean veggie soup,16,55,187.5,4.0,19.0,13.0,18.0,...,2.944439,2.564949,2.890372,0.693147,2.302585,469903,2010-04-15,"Nicely spiced black bean soup, it certainly wo...",5,"this comfort-filled, nourishing soup is my mod..."
63915,89503,meat,crispy baked chicken strips,7,30,617.5,11.0,2.0,20.0,78.0,...,0.693147,2.995732,4.356709,2.079442,3.433987,125388,2010-04-15,"These were really good, and so easy to mix up ...",5,these are delicious! and much more cost-effect...
63916,418966,vegetables,tossed caprese salad,9,20,243.9,30.0,12.0,16.0,26.0,...,2.484907,2.772589,3.258097,3.737670,0.000000,464080,2010-04-15,Very good indeed. Great use for the Trader Jo...,5,a tossed salad version of a classic italian ap...
63917,414998,beverages,single serve hot chocolate fast,4,3,187.3,5.0,103.0,10.0,23.0,...,4.634729,2.302585,3.135494,1.386294,2.197225,201581,2010-04-15,What a great treat on a chilly morning. Easy t...,5,you're out of all the usual suspects and someb...
63918,161941,meat,smoked pork shoulder smoker or pit,15,90,735.0,7.0,564.0,738.0,18.0,...,6.335054,6.603944,2.890372,1.386294,4.077537,804550,2010-04-15,Delicious smokehouse flavor. When my DD and I ...,5,this recipe was given to me by a friend of my ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85232,289268,soup,mediterranean kale white bean soup with sausage,14,55,330.1,26.0,13.0,40.0,38.0,...,2.564949,3.688879,3.637586,3.178054,2.197225,198154,2018-12-02,I had to make small adjustments according to w...,5,pulled from fine cooking magazine (feb/mar. 20...
85233,140486,desserts,basic vanilla frosting,4,5,1162.8,37.0,940.0,9.0,2.0,...,6.845880,2.197225,0.693147,4.330733,4.382027,227652,2018-12-05,This was incredibly simple and easy to put tog...,5,this is simple to make and has basic ingredien...
85234,121066,pasta,garlic shrimp pasta,10,30,408.1,14.0,8.0,20.0,41.0,...,2.079442,2.995732,3.713572,3.218876,2.944439,198154,2018-12-09,I doubled the garlic and felt that it could us...,4,this is my hubby's specialty! this is one of t...
85235,19653,desserts,roll out cookies,6,25,1870.6,147.0,402.0,50.0,43.0,...,5.996452,3.912023,3.761200,5.690359,4.343805,227652,2018-12-10,These cookies were wonderful. I've had this in...,5,my favorite cookie recipe to use with cookie c...


## Check all users in test set has reviewed at least 10 recipes
Can compare Top 10 recipes in the testing stage of the recommender system

In [65]:
review_cnt = test_set['user_id'].value_counts()
review_cnt

266635    906
895132    675
865936    542
198154    537
844554    509
         ... 
124249      3
141293      3
52543       2
49304       2
21752       1
Name: user_id, Length: 117, dtype: int64

In [66]:
# check if each recipe is only reviewed once by each user
cnt_check = test_set.groupby(['user_id', 'recipe_id']).count()
cnt_check

category  name  n_ingredients  minutes  calories  \
user_id recipe_id                                                     
1535    10125             1     1              1        1         1   
        10465             1     1              1        1         1   
        16533             1     1              1        1         1   
        16901             1     1              1        1         1   
        19424             1     1              1        1         1   
...                     ...   ...            ...      ...       ...   
895132  517502            1     1              1        1         1   
        517904            1     1              1        1         1   
        518260            1     1              1        1         1   
        518983            1     1              1        1         1   
        522871            1     1              1        1         1   

                   total fat (PDV)  sugar (PDV)  sodium (PDV)  protein (PDV)  \
user_id recipe_id                                                              
1535    10125                    1            1             1              1   
        10465                    1            1             1              1   
        16533                    1            1             1              1   
        16901                    1            1             1              1   
        19424                    1            1             1              1   
...                            ...          ...           ...            ...   
895132  517502                   1            1             1              1   
        517904                   1            1             1              1   
        518260                   1            1             1              1   
        518983                   1            1             1              1   
        522871                   1            1             1              1   

                   saturated fat (PDV)  ...  log fat  log sugar  log sodium  \
user_id recipe_id                       ...                                   
1535    10125                        1  ...        1          1           1   
        10465                        1  ...        1          1           1   
        16533                        1  ...        1          1           1   
        16901                        1  ...        1          1           1   
        19424                        1  ...        1          1           1   
...                                ...  ...      ...        ...         ...   
895132  517502                       1  ...        1          1           1   
        517904                       1  ...        1          1           1   
        518260                       1  ...        1          1           1   
        518983                       1  ...        1          1           1   
        522871                       1  ...        1          1           1   

                   log protein  log sat fat  log carbs  date  review  rating  \
user_id recipe_id                                                              
1535    10125                1            1          1     1       1       1   
        10465                1            1          1     1       1       1   
        16533                1            1          1     1       1       1   
        16901                1            1          1     1       1       1   
        19424                1            1          1     1       1       1   
...                        ...          ...        ...   ...     ...     ...   
895132  517502               1            1          1     1       1       1   
        517904               1            1          1     1       1       1   
        518260               1            1          1     1       1       1   
        518983               1            1          1     1       1       1   
        522871               1            1          1     1      

In [67]:
# since the number is equal to the number of in the test set, each user only review the recipe once
cnt_check.sum() 

category               20700
name                   20700
n_ingredients          20700
minutes                20700
calories               20700
total fat (PDV)        20700
sugar (PDV)            20700
sodium (PDV)           20700
protein (PDV)          20700
saturated fat (PDV)    20700
carbohydrates (PDV)    20700
log calories           20700
log fat                20700
log sugar              20700
log sodium             20700
log protein            20700
log sat fat            20700
log carbs              20700
date                   20700
review                 20700
rating                 20700
description            20700
dtype: int64

In [68]:
less10 = []
for i in review_cnt.index:
    if review_cnt[i] < 10:
        less10.append(i)
        
print('no. of users (< 10 recipes): ', len(less10))
print(less10)

no. of users (< 10 recipes):  12
[102058, 121690, 431813, 18391, 52282, 59064, 27783, 124249, 141293, 52543, 49304, 21752]


In [69]:
# Drop user that reviewed < 10 recipes
for user in less10:
    test_set = test_set[test_set['user_id'] != user]

In [70]:
test_set

,recipe_id,category,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),...,log sugar,log sodium,log protein,log sat fat,log carbs,user_id,date,review,rating,description
63914,327626,soup,miracle black bean veggie soup,16,55,187.5,4.0,19.0,13.0,18.0,...,2.944439,2.564949,2.890372,0.693147,2.302585,469903,2010-04-15,"Nicely spiced black bean soup, it certainly wo...",5,"this comfort-filled, nourishing soup is my mod..."
63915,89503,meat,crispy baked chicken strips,7,30,617.5,11.0,2.0,20.0,78.0,...,0.693147,2.995732,4.356709,2.079442,3.433987,125388,2010-04-15,"These were really good, and so easy to mix up ...",5,these are delicious! and much more cost-effect...
63916,418966,vegetables,tossed caprese salad,9,20,243.9,30.0,12.0,16.0,26.0,...,2.484907,2.772589,3.258097,3.737670,0.000000,464080,2010-04-15,Very good indeed. Great use for the Trader Jo...,5,a tossed salad version of a classic italian ap...
63917,414998,beverages,single serve hot chocolate fast,4,3,187.3,5.0,103.0,10.0,23.0,...,4.634729,2.302585,3.135494,1.386294,2.197225,201581,2010-04-15,What a great treat on a chilly morning. Easy t...,5,you're out of all the usual suspects and someb...
63918,161941,meat,smoked pork shoulder smoker or pit,15,90,735.0,7.0,564.0,738.0,18.0,...,6.335054,6.603944,2.890372,1.386294,4.077537,804550,2010-04-15,Delicious smokehouse flavor. When my DD and I ...,5,this recipe was given to me by a friend of my ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85232,289268,soup,mediterranean kale white bean soup with sausage,14,55,330.1,26.0,13.0,40.0,38.0,...,2.564949,3.688879,3.637586,3.178054,2.197225,198154,2018-12-02,I had to make small adjustments according to w...,5,pulled from fine cooking magazine (feb/mar. 20...
85233,140486,desserts,basic vanilla frosting,4,5,1162.8,37.0,940.0,9.0,2.0,...,6.845880,2.197225,0.693147,4.330733,4.382027,227652,2018-12-05,This was incredibly simple and easy to put tog...,5,this is simple to make and has basic ingredien...
85234,121066,pasta,garlic shrimp pasta,10,30,408.1,14.0,8.0,20.0,41.0,...,2.079442,2.995732,3.713572,3.218876,2.944439,198154,2018-12-09,I doubled the garlic and felt that it could us...,4,this is my hubby's specialty! this is one of t...
85235,19653,desserts,roll out cookies,6,25,1870.6,147.0,402.0,50.0,43.0,...,5.996452,3.912023,3.761200,5.690359,4.343805,227652,2018-12-10,These cookies were wonderful. I've had this in...,5,my favorite cookie recipe to use with cookie c...


In [71]:
review_cnt2 = test_set['user_id'].value_counts()
review_cnt2

266635    906
895132    675
865936    542
198154    537
844554    509
         ... 
20754      12
56112      12
130663     12
13483      10
37305      10
Name: user_id, Length: 105, dtype: int64

In [72]:
# convert to csv
train_set.to_csv('./Dataset/train_dataset_all.csv', index=False)
test_set.to_csv('./Dataset/test_dataset_all.csv', index=False)